In [54]:
%reload_ext autoreload
%autoreload 2

import glob
import os, gc
import numpy as numpy
import pandas as pd
import scipy as sp
import datatable as dt
from collections import defaultdict
from tqdm.notebook import tqdm
from sklearn.utils import shuffle
from sklearn.metrics import r2_score
from numba import njit
from utils import *

from IPython.display import clear_output

# TF
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

In [55]:
N_FOLD = 5
N_MINS = 5
MIN_SIZE = 600 // N_MINS

SOL_NAME = '501-NN-test'
DATA_NAME = '501'
mkdir(f'./models/{SOL_NAME}/')

In [56]:
# CONSTANT
MEAN = -5.762330803300896
STD = 0.6339307835941186
EPS = 1e-9

In [57]:
# get ids
list_stock_id = get_stock_id()
list_time_id = get_time_id()

# Functions

In [58]:
def transform_target(target):
    return (np.log(target + EPS) - MEAN) / STD

def inverse_target(target):
    return np.exp(MEAN + STD * target) - EPS

def np_rmspe(y_true, y_pred):
    y_true = inverse_target(y_true)
    y_pred = inverse_target(y_pred)
    y_pred = np.clip(y_pred, 0, None)
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

def mspe_loss(y_true, y_pred):
    y_true = K.exp(MEAN + STD * y_true) - EPS
    y_pred = K.exp(MEAN + STD * y_pred) - EPS
    y_pred = K.clip(y_pred, 0, None)
    return K.sqrt(K.mean(K.square((y_true - y_pred) / y_true)))

def rmspe_keras(y_true, y_pred):
    return K.sqrt(K.mean(K.square((y_true - y_pred) / y_true)))

In [59]:
def create_mlp(num_columns, num_labels, hidden_units, dropout_rates, learning_rate):
    inp = tf.keras.layers.Input(shape=(num_columns,))
    x = tf.keras.layers.BatchNormalization()(inp)
    for i in range(len(hidden_units)):
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
    # Output layer
    x = tf.keras.layers.Dense(num_labels)(x)
    out = tf.keras.layers.Activation('linear')(x)

    model = tf.keras.models.Model(inputs=inp, outputs=out)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=[mspe_loss],
    )
    return model

# Loading data

In [60]:
# train
df_train = dt.fread(f'./dataset/train_{DATA_NAME}_NN_noclip.csv').to_pandas()
fea_cols = [f for f in df_train.columns if f.startswith('B_') or f.startswith('T_') or f.startswith('Z_')]
# result
df_result = dt.fread('./dataset/train.csv').to_pandas()
df_result = gen_row_id(df_result)

In [61]:
fea_cols_TA = [f for f in fea_cols if 'min_' not in f]
df_time_mean = df_train.groupby('time_id')[fea_cols_TA].mean()
df_time_mean.columns = [f'{c}_TA_mean' for c in df_time_mean.columns]
df_time_mean = df_time_mean.reset_index()
df_train = df_train.merge(df_time_mean, on='time_id', how='left')

del df_time_mean
gc.collect()

269

In [62]:
df_train['target'] = transform_target(df_train['target'])
df_train = gen_row_id(df_train)
df_train = add_time_fold(df_train, N_FOLD)

# Evaluation

In [63]:
def add_time_stats(df_train):
    time_cols = [f for f in df_train.columns if f.endswith('_time')]
    df_gp_stock = df_train.groupby('stock_id')
    #
    df_stats = df_gp_stock[time_cols].mean().reset_index()
    df_stats.columns = ['stock_id'] + [f'{f}_mean' for f in time_cols]
    df_train = df_train.merge(df_stats, on=['stock_id'], how='left')
    #
    df_stats = df_gp_stock[time_cols].std().reset_index()
    df_stats.columns = ['stock_id'] + [f'{f}_std' for f in time_cols]
    df_train = df_train.merge(df_stats, on=['stock_id'], how='left')
    #
    df_stats = df_gp_stock[time_cols].skew().reset_index()
    df_stats.columns = ['stock_id'] + [f'{f}_skew' for f in time_cols]
    df_train = df_train.merge(df_stats, on=['stock_id'], how='left')
    #
    df_stats = df_gp_stock[time_cols].min().reset_index()
    df_stats.columns = ['stock_id'] + [f'{f}_min' for f in time_cols]
    df_train = df_train.merge(df_stats, on=['stock_id'], how='left')
    #
    df_stats = df_gp_stock[time_cols].max().reset_index()
    df_stats.columns = ['stock_id'] + [f'{f}_max' for f in time_cols]
    df_train = df_train.merge(df_stats, on=['stock_id'], how='left')
    #
    df_stats = df_gp_stock[time_cols].quantile(0.25).reset_index()
    df_stats.columns = ['stock_id'] + [f'{f}_q1' for f in time_cols]
    df_train = df_train.merge(df_stats, on=['stock_id'], how='left')
    #
    df_stats = df_gp_stock[time_cols].quantile(0.50).reset_index()
    df_stats.columns = ['stock_id'] + [f'{f}_q2' for f in time_cols]
    df_train = df_train.merge(df_stats, on=['stock_id'], how='left')
    #
    df_stats = df_gp_stock[time_cols].quantile(0.75).reset_index()
    df_stats.columns = ['stock_id'] + [f'{f}_q3' for f in time_cols]
    df_train = df_train.merge(df_stats, on=['stock_id'], how='left')
    return df_train

In [64]:
batch_size = 1024
# hidden_units = [256]*4
hidden_units = [64, 32, 16]
dropout_rates = 0
learning_rate = 6e-3
epochs = 1000

list_rmspe = [1 for _ in range(N_FOLD)]

In [65]:
n_trials = 5
for _ in range(n_trials):
    for i_fold in range(N_FOLD):
        gc.collect()
        df_tr = df_train.loc[df_train.fold!=i_fold]
        df_te = df_train.loc[df_train.fold==i_fold]
        df_tr = add_time_stats(df_tr)
        df_te = add_time_stats(df_te)


        fea_cols = [f for f in df_tr.columns if f.startswith('B_') or f.startswith('T_') or f.startswith('Z_')]

        X_train = df_tr[fea_cols].values
        y_train = df_tr[['target']].values
        X_test = df_te[fea_cols].values
        y_test = df_te[['target']].values

        idx_test = df_train.loc[df_train.fold==i_fold].index
        print(f'Fold {i_fold+1}/{N_FOLD}', X_train.shape, X_test.shape)


        # Callbacks
        ckp_path = f'./models/{SOL_NAME}/model_{i_fold}.hdf5'
        rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=8, min_delta=1e-5, verbose=2)
        es = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=30, restore_best_weights=True, verbose=2)

        model = create_mlp(X_train.shape[1], 1, hidden_units, dropout_rates, learning_rate) 
        history = model.fit(X_train, y_train,
            epochs=epochs,
            validation_data=(X_test, y_test),
            validation_batch_size=len(y_test),
            batch_size=batch_size,
            verbose=2,
            callbacks=[rlr, es]
        ) 
        # model = tf.keras.models.load_model(ckp_path, custom_objects={'mspe_loss': mspe_loss})

        y_pred = model.predict(X_test, batch_size=1024)

        curr_rmspe = np_rmspe(y_test, y_pred)
        if curr_rmspe < list_rmspe[i_fold]:
            model.save(ckp_path)
            list_rmspe[i_fold] = curr_rmspe
            # generate and save preds
            df_result.loc[idx_test, 'pred'] = inverse_target(y_pred).clip(0, None)
        clear_output()
        print(list_rmspe)
        calc_metric(df_result.fillna(0))
    # break

[0.20867723214772477, 0.21223763725721875, 0.20974995911090152, 0.21065104055433476, 0.217691254517265]
   R2: 0.8365
RMSPE: 0.2118


KeyboardInterrupt: 

In [ ]:
df_result.to_csv(f'./results/{SOL_NAME}.csv', index=False)